In [2]:
import re
import tweepy
import pandas as pd
from tweepy import OAuthHandler
import yaml
from tabulate import tabulate

# from http.client import IncompleteRead
# import threading 
# import datetime 
    
    
with open('credential3.yaml') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    
consumer_key = config['consumer_key']
consumer_secret = config['consumer_secret']
access_token = config['access_token']
access_secret = config['access_secret']
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#### Set date

In [98]:
date = '2020-11-04'
start = '2020-11-04 00:00:00'
end = '2020-11-05 00:00:00'

In [93]:
# date = '2020-11-03'
# df = pd.read_csv("./data/{}.csv".format(date))
# print("{}.csv".format(date))

2020-11-03.csv


#### Collect tweets with hashtag `#30mapchallenges`

In [99]:
def collect_tweets(search_words, since):
    #tweets
    tweet_id = []
    tweet_text = []
    language = []
    tweet_created_at = []
    retweet_count = []
    favorite_count= []
    hashtags= []
    #users
    user_id = []
    user_name = []
    user_screen_name = []
    user_loc = []
    user_follower = []
    friends_count = []
    #media
    media = []
    for tweet in tweepy.Cursor(api.search,
                               q=search_words,
                               since= since, 
                               include_entities=True, 
                               tweet_mode="extended").items():
        #tweets
        tweet_id.append(tweet.id)
        tweet_text.append(tweet.full_text)
        language.append(tweet.lang)
        tweet_created_at.append(tweet.created_at)
        retweet_count.append(tweet.retweet_count)
        favorite_count.append(tweet.favorite_count)
        try:
            hashtags.append(tweet.entities.get('hashtags', []))
        except KeyError:
            pass
#         hashtags.append(tweet.entities.hashtags)
        #users
        user_id.append(tweet.user.id)
        user_name.append(tweet.user.name)
        user_screen_name.append(tweet.user.screen_name)
        user_loc.append(tweet.user.location)
        user_follower.append(tweet.user.followers_count)
        friends_count.append(tweet.user.friends_count)
        #media
        try:
            media.append(tweet.entities.get('media')[0].get('media_url'))
#             print(tweet.entities.get('media'))
        except Exception as e:
            media.append(None)
        #print([tweet.user.name,tweet.created_at,tweet.text])
    df = pd.DataFrame({'tweet_id':tweet_id,
                        'tweet_text':tweet_text,
                        'language':language,
                        'tweet_created_at':tweet_created_at,
                        'retweet':retweet_count,
                        'favorite':favorite_count,
                        'hashtags':hashtags,
                        'user_id':user_id,
                        'user_name':user_name,
                        'user':user_screen_name,
                        'user_loc':user_loc,
                        'user_follower':user_follower,
                        'friends_count':friends_count,
                        'media':media})
            
    return df

In [102]:
search_words = "#30DayMapChallenge -filter:retweets"
since = "2020-10-30"

df = collect_tweets(search_words, since)

df = df[(df.tweet_created_at >= start) & 
   (df.tweet_created_at <= end)]

# df1 = df[df['tweet_text'].str.contains("points") | df['tweet_text'].str.contains("day 1")]

df = df[df['media'].notnull()]
df['tweet_url'] = df['tweet_text'].str.extract('(?P<url>https?://[^\s]+)', expand=False).str.strip()


df.to_csv("./data/{}.csv".format(date))

In [103]:
df

,tweet_id,tweet_text,language,tweet_created_at,retweet,favorite,hashtags,user_id,user_name,user,user_loc,user_follower,friends_count,media,tweet_url
174,1324139193543569410,Day 4 of the #30DayMapChallenge with @OmniSci:...,en,2020-11-04 23:59:19,1,6,"[{'text': '30DayMapChallenge', 'indices': [13,...",623327023,Anton1o,AFCotroneo,,187,1302,http://pbs.twimg.com/media/EmBJfXaXgAIPp3i.jpg,https://t.co/uTi98Iv8nr
175,1324138830455214080,Reef and MPAs for the Pacific. #30DayMapChalle...,en,2020-11-04 23:57:53,1,9,"[{'text': '30DayMapChallenge', 'indices': [31,...",4273148897,Carrol Chan,cmhc23,Fiji,435,387,http://pbs.twimg.com/media/EmBJHvuUYAENptQ.jpg,https://t.co/GSZKv2GRN7
176,1324138467555770369,"Early, but day 5 of Early #30DayMapChallenge, ...",en,2020-11-04 23:56:26,5,6,"[{'text': '30DayMapChallenge', 'indices': [26,...",1280909342745800710,Chris Lyons,ChrisLyons26,"Shelbyville, KY",198,2589,http://pbs.twimg.com/media/EmBI6rPX0AAzECx.jpg,https://t.co/rTo2KhCABL
177,1324137155636187136,#30DayMapChallenge Day 4 Hexagons - tenuous li...,en,2020-11-04 23:51:13,1,1,"[{'text': '30DayMapChallenge', 'indices': [0, ...",1091711968401154048,Brian Price,BrianSosban,"Burscough, Lancashire, UK",3,36,http://pbs.twimg.com/media/EmBHrSIX0AEbM4Q.jpg,https://t.co/EFCnhIKz4y
178,1324135805779148802,#30DayMapChallenge Day 4... HEXAGONS\n\nNarwha...,en,2020-11-04 23:45:52,1,6,"[{'text': '30DayMapChallenge', 'indices': [0, ...",1200892211996635139,Danae Kontou,DanaeKontou,,10,86,http://pbs.twimg.com/media/EmBFCLXWkAkTMZQ.png,https://t.co/QuQzepAq1k
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,1323779050066857984,"Day 3, Polygons . Continuing my Vermont theme ...",en,2020-11-04 00:08:14,1,2,"[{'text': '30DayMapChallenge', 'indices': [70,...",721829381207887872,David Fox,ReynardSubtil,"Vermont, USA",61,163,http://pbs.twimg.com/media/El8B5VVXgAEAKPE.jpg,https://t.co/zvFDvomebq
540,1323777937208987648,For Day 3 of the #30DayMapChallenge (Polygons)...,en,2020-11-04 00:03:49,1,18,"[{'text': '30DayMapChallenge', 'indices': [17,...",1146772513051041792,Samara Ebinger,EbingerSamara,"New Hampshire, USA",168,228,http://pbs.twimg.com/media/El7_799X0AE-rM7.png,https://t.co/DTPCqJvCrd
541,1323777436140580864,#30DayMapChallenge Day 4 | Hexagons\n\nA recen...,en,2020-11-04 00:01:50,1,9,"[{'text': '30DayMapChallenge', 'indices': [0, ...",2614024302,Rob Radburn,robradburn,"Leicester, UK",3181,749,http://pbs.twimg.com/media/El8AjsaWMAAnCkP.jpg,https://t.co/W1Wf05G5GA
542,1323777363050700800,Day 4 - Hexagons #30DayMapChallenge\nGone topi...,en,2020-11-04 00:01:32,1,8,"[{'text': '30DayMapChallenge', 'indices': [17,...",996835888813494272,Catherine Schroder,cj_maps,"Scotland, United Kingdom",19,22,http://pbs.twimg.com/media/El8AftWX0AgDZuM.jpg,https://t.co/Q6yGIBcVoU


### late entry (only for day 2)

In [7]:
# df[df['tweet_text'].str.contains('points', case=False) & 
#    df['tweet_text'].str.contains('day 1', case=False)].to_csv("./data/{}_late.csv".format(date))

In [8]:
# df = df.drop(df[df['tweet_text'].str.contains('points', case=False) & 
#                 df['tweet_text'].str.contains('day 1', case=False)].index)

---

## Overview

In [16]:
df[['retweet','favorite', 'user_follower', 'friends_count']].describe()

,retweet,favorite,user_follower,friends_count
count,393.000000,393.000000,393.000000,393.000000
mean,3.249364,16.778626,1409.720102,662.134860
std,5.836978,36.911358,5689.031444,857.301559
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,4.000000,77.000000,158.000000
50%,2.000000,8.000000,280.000000,335.000000
75%,4.000000,18.000000,876.000000,780.000000
max,91.000000,624.000000,80333.000000,5454.000000


## Top favorited/retweeted `#30mapchallenges` post

In [82]:
def sort_map(df, sortby, num):
    print(date +'...\n')
    df = df.sort_values(by='favorite', ascending=False).head(num)
    df['user'] = '**[@' + df['user'] +']('+ df['tweet_url']+')**'
    df['media'] = '![](' + df['media'] +')'
    df = df[['user', 'retweet', 'favorite', 'media']].set_index("user")
    print(tabulate(df, headers='keys', tablefmt='github'))

In [85]:
def tweet_shortcode(df, sortby, num):
    print(date +'...\n')
    df = df.sort_values(by=sortby, ascending=False).head(num)
    print("## Top `{}` maps for Day {}".format(sortby, date[-2:]))
    for i in df['tweet_id']:
        print('{{< tweet ' + str(i) +' >}}')

In [104]:
sort_map(df, 'favorite', 15)

2020-11-04...

| user                                            |   retweet |   favorite | media                                                                                        |
|-------------------------------------------------|-----------|------------|----------------------------------------------------------------------------------------------|
| **[@rajbhagatt](https://t.co/DU5EDua1NX)**      |        21 |        152 | ![](http://pbs.twimg.com/media/El_Lj7uVgAAUooh.jpg)                                          |
| **[@Artisans_Cartos](https://t.co/qLduuBN5UC)** |        26 |        125 | ![](http://pbs.twimg.com/tweet_video_thumb/EmAE9UVXEAEeKaO.jpg)                              |
| **[@darenw](https://t.co/CLGTIwbNwj)**          |         9 |         98 | ![](http://pbs.twimg.com/ext_tw_video_thumb/1323991938035523584/pu/img/OV5MM55om4WRHcY8.jpg) |
| **[@cstats1](https://t.co/y9yZDTcGmj)**         |        17 |         87 | ![](http://pbs.twimg.com/ext_tw_video_thumb/1324

In [105]:
sort_map(df, 'retweet', 15)

2020-11-04...

| user                                            |   retweet |   favorite | media                                                                                        |
|-------------------------------------------------|-----------|------------|----------------------------------------------------------------------------------------------|
| **[@rajbhagatt](https://t.co/DU5EDua1NX)**      |        21 |        152 | ![](http://pbs.twimg.com/media/El_Lj7uVgAAUooh.jpg)                                          |
| **[@Artisans_Cartos](https://t.co/qLduuBN5UC)** |        26 |        125 | ![](http://pbs.twimg.com/tweet_video_thumb/EmAE9UVXEAEeKaO.jpg)                              |
| **[@darenw](https://t.co/CLGTIwbNwj)**          |         9 |         98 | ![](http://pbs.twimg.com/ext_tw_video_thumb/1323991938035523584/pu/img/OV5MM55om4WRHcY8.jpg) |
| **[@cstats1](https://t.co/y9yZDTcGmj)**         |        17 |         87 | ![](http://pbs.twimg.com/ext_tw_video_thumb/1324

In [106]:
tweet_shortcode(df, 'favorite', 15)

2020-11-04...

## Top `favorite` maps for Day 04
{{< tweet 1324000845134880769 >}}
{{< tweet 1324064281382572034 >}}
{{< tweet 1323992133334781952 >}}
{{< tweet 1324114711516950529 >}}
{{< tweet 1324009073692479489 >}}
{{< tweet 1324113530661097477 >}}
{{< tweet 1324004099965263874 >}}
{{< tweet 1323994953245184000 >}}
{{< tweet 1324018953895747584 >}}
{{< tweet 1324054397421236231 >}}
{{< tweet 1323905416598880258 >}}
{{< tweet 1323898023424962561 >}}
{{< tweet 1323951970323959808 >}}
{{< tweet 1323884771739619329 >}}
{{< tweet 1323971979855220739 >}}


In [107]:
tweet_shortcode(df, 'retweet', 15)

2020-11-04...

## Top `retweet` maps for Day 04
{{< tweet 1324064281382572034 >}}
{{< tweet 1324000845134880769 >}}
{{< tweet 1324009073692479489 >}}
{{< tweet 1324018953895747584 >}}
{{< tweet 1324114711516950529 >}}
{{< tweet 1324004099965263874 >}}
{{< tweet 1323905416598880258 >}}
{{< tweet 1324054397421236231 >}}
{{< tweet 1324051285671350275 >}}
{{< tweet 1323917536719740930 >}}
{{< tweet 1324055479086174208 >}}
{{< tweet 1324113530661097477 >}}
{{< tweet 1323796374757003265 >}}
{{< tweet 1323892285956870145 >}}
{{< tweet 1323992133334781952 >}}


### Late entry

In [25]:
# df_late = pd.read_csv("./data/{}_late.csv".format(date))
# print("{}_late.csv".format(date))
# print("Add to day 1...")

2020-11-02_late.csv
Add to day 1...


In [27]:
# df_late_fav = df_late.sort_values(by='favorite', ascending=False).head(5)
# df_late_fav = df_late_fav[['user', 'retweet', 'favorite', 'media']]
# df_late_fav['user'] = '**[@' + df_late['user'] +']('+ df_late['tweet_url']+')**'
# df_late_fav['media'] = '![](' + df_late['media'] +')'

In [28]:
print(tabulate(df_late_fav, headers='keys', tablefmt='github'))

|    | user                                            |   retweet |   favorite | media                                               |
|----|-------------------------------------------------|-----------|------------|-----------------------------------------------------|
| 42 | **[@darenw](https://t.co/rZ2gtbJmI4)**          |         4 |         95 | ![](http://pbs.twimg.com/media/Elyc4d4XUAEs09h.jpg) |
| 50 | **[@gonzalotalavera](https://t.co/I6UNoGYVOx)** |        22 |         60 | ![](http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2.jpg) |
| 53 | **[@PratapVardhan](https://t.co/WTT8WM17qy)**   |         4 |         43 | ![](http://pbs.twimg.com/media/Elx5722WkAEOOMf.jpg) |
|  2 | **[@tomasral](https://t.co/khysneCwjM))**       |         5 |         35 | ![](http://pbs.twimg.com/media/El2FR9gXUAAU3Uf.jpg) |
| 54 | **[@BZgeo](https://t.co/LavH2067O2)**           |         3 |         26 | ![](http://pbs.twimg.com/media/Elx0-fPXIAc9JGJ.png) |


In [29]:
# df_late_rt = df_late.sort_values(by='retweet', ascending=False).head(5)
# df_late_rt = df_late_rt[['user', 'retweet', 'favorite', 'media']]
# df_late_rt['user'] = '**[@' + df_late['user'] +']('+ df_late['tweet_url']+')**'
# df_late_rt['media'] = '![](' + df_late['media'] +')'

In [30]:
# print(tabulate(df_late_rt, headers='keys', tablefmt='github'))

|    | user                                            |   retweet |   favorite | media                                               |
|----|-------------------------------------------------|-----------|------------|-----------------------------------------------------|
| 50 | **[@gonzalotalavera](https://t.co/I6UNoGYVOx)** |        22 |         60 | ![](http://pbs.twimg.com/media/ElyMGZFWkAA7Fx2.jpg) |
| 24 | **[@neocartocnrs](https://t.co/1GcTGraT9T)**    |         8 |         14 | ![](http://pbs.twimg.com/media/Elz2OvNXEAAItfe.jpg) |
|  2 | **[@tomasral](https://t.co/khysneCwjM))**       |         5 |         35 | ![](http://pbs.twimg.com/media/El2FR9gXUAAU3Uf.jpg) |
| 57 | **[@nurussadad](https://t.co/z4HaIfQMBi)**      |         4 |          7 | ![](http://pbs.twimg.com/media/ElxuzgpU0AEIbdT.png) |
|  7 | **[@geologyjesse](https://t.co/dR68mE4zhD)**    |         4 |         14 | ![](http://pbs.twimg.com/media/El1vroKWoAEozEk.jpg) |


### Send tweet via API

In [48]:
status = 'A github repo of monitoring tweets feeds with wonderful maps. Github: https://github.com/hn303/30DayMapChallenge-Bot'

In [45]:
imagePath = "screenshot/github.com_hn303_30DayMapChallenge-Bot.png"

In [49]:
api.update_status(status)

Status(_api=<tweepy.api.API object at 0x000001D700B3FC88>, _json={'created_at': 'Wed Nov 04 13:14:23 +0000 2020', 'id': 1323976890621792256, 'id_str': '1323976890621792256', 'text': 'A github repo of monitoring tweets feeds with wonderful maps. **Github**:https://t.co/J1NHDOPpdT', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/J1NHDOPpdT', 'expanded_url': 'https://github.com/hn303/30DayMapChallenge-Bot', 'display_url': 'github.com/hn303/30DayMap…', 'indices': [73, 96]}]}, 'source': '<a href="https://www.landecon.cam.ac.uk/research/research-centres/lisa/members" rel="nofollow">Untitle_APP3</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1078706606488068096, 'id_str': '1078706606488068096', 'name': 'Stan', 'screen_name': 'Stan_CamLE', 'location': '', 'description': '', 'url': None, 'entit

In [46]:
api.update_with_media(imagePath, status)

Status(_api=<tweepy.api.API object at 0x000001D700B3FC88>, _json={'created_at': 'Wed Nov 04 12:58:56 +0000 2020', 'id': 1323973002543976450, 'id_str': '1323973002543976450', 'text': 'A github repo of monitoring tweets feeds with wonderful maps. https://t.co/J1NHDOPpdT https://t.co/HwA2aVTax1', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/J1NHDOPpdT', 'expanded_url': 'https://github.com/hn303/30DayMapChallenge-Bot', 'display_url': 'github.com/hn303/30DayMap…', 'indices': [62, 85]}], 'media': [{'id': 1323972999717003266, 'id_str': '1323972999717003266', 'indices': [86, 109], 'media_url': 'http://pbs.twimg.com/media/El-ybbRXIAICiB1.jpg', 'media_url_https': 'https://pbs.twimg.com/media/El-ybbRXIAICiB1.jpg', 'url': 'https://t.co/HwA2aVTax1', 'display_url': 'pic.twitter.com/HwA2aVTax1', 'expanded_url': 'https://twitter.com/Stan_CamLE/status/1323973002543976450/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 